# Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:

    оценка 2 и меньше - низкий рейтинг
    оценка 4 и меньше - средний рейтинг
    оценка 4.5 и 5 - высокий рейтинг
    
Результат классификации запишите в столбец class

In [2]:
import pandas as pd

In [4]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
movies_list = ratings.merge(movies, on = 'movieId', how = 'left')

def rating_count(row):
    if row['rating'] <= 2:
        return 'Low rating'
    if row['rating'] <= 4:
        return 'Average rating'
    if row['rating'] >= 4.5:
        return 'High rating'
    
movies_list['class'] = movies_list.apply(rating_count, axis = 1)
movies_list.head(10)

,userId,movieId,rating,timestamp,title,genres,class
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,Average rating
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,Average rating
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,Average rating
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,Low rating
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,Average rating
5,1,1263,2.0,1260759151,"Deer Hunter, The (1978)",Drama|War,Low rating
6,1,1287,2.0,1260759187,Ben-Hur (1959),Action|Adventure|Drama,Low rating
7,1,1293,2.0,1260759148,Gandhi (1982),Drama,Low rating
8,1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller,Average rating
9,1,1343,2.0,1260759131,Cape Fear (1991),Thriller,Low rating


# Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

In [7]:
data = pd.read_csv('ml-latest-small/keywords.csv')

geo_data = {
'Центр': ['москва', 'тула', 'ярославль', 'казань'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

def find_region(element):
    for name_of_region, city_of_region in geo_data.items():
        for city in city_of_region:
            if city in element:
                return name_of_region
    return 'undefined'

data['region'] = data['keyword'].apply(find_region)
data.head(373)

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined
...,...,...,...
368,футбол на куличках,433116,undefined
369,порно инцест,504714,undefined
370,авито ру санкт петербург,425134,Северо-Запад
371,игры онлайн,588264,undefined


# Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [17]:
df_ratings = pd.read_csv('ratings.csv')
df_movies = pd.read_csv('movies.csv')

df_ratings_mean = df_ratings.groupby('movieId').mean()
# средний рейтинг с группировкой по столбцу

years = range(1950, 2011)
#создаем список с годами для нахождения года в описании фильма и вынесении полученного года в отдельный столбец year

def production_year(title):
    '''для поиска года выпуска фильма- ищем вхождения списка с годами() в контекст строки title'''
    year = 1900
    for item in years:
        i = str(item)
        if i in title:
            year = item
    return year

In [19]:
df_movies['year'] = df_movies['title'].apply(production_year)
#обогощаем датафрейм с рейтингм данными с датафрейма с фильиами (ключевое поле movieId)

df_merged = pd.merge(df_ratings_mean, df_movies, left_index = True, right_on = 'movieId' )
#оставляем для отчета только 2 столбца,группируем по годам, вычисляея средний рейтинг, сортируем по убыванию столбец "Рейтинг"

df_merged[['rating','year']].groupby('year').mean().sort_values('rating', ascending=False).head(20)

,rating
year,
1963,3.228859
1961,3.208200
1962,3.194438
1953,3.189489
1950,3.178434
1964,3.178317
1965,3.152148
1952,3.145530
1955,3.130861
